In [6]:
import os
import pandas as pd
import numpy as np

In [7]:
maindir = "./results-g512"
execution="CPU"

In [8]:
def get_min_max_mean(df, column, start_time, end_time):
    df_i = df[(df['TIMESTAMP'] >= start_time) & (df['TIMESTAMP'] <= end_time)]
    min_value = df_i[column].min()
    max_value = df_i[column].max()
    mean_value = df_i[column].mean()    
    return min_value, max_value, mean_value
  
def add_min_max_mean_columns(df, column, index, min_value, max_value, mean_value):
    df.loc[index, f'{column}_MIN'] = min_value
    df.loc[index, f'{column}_MAX'] = max_value
    df.loc[index, f'{column}_MEAN'] = mean_value

def sumarise_df_cpu(df_cpu, df_mem_cpu, df_timestamp):
    df_cpu['TIMESTAMP'] = df_cpu['TIMESTAMP'].astype(int)
    df_mem_cpu['TIMESTAMP'] = df_mem_cpu['TIMESTAMP'].astype(int)
    df_timestamp['START_TIME'] = df_timestamp['START_TIME'].astype(str).str[:10].astype(int)
    df_timestamp['END_TIME'] = df_timestamp['END_TIME'].astype(str).str[:10].astype(int)
  
    for i, r in df_timestamp.iterrows():
        start_time = r['START_TIME']
        end_time = r['END_TIME']
        
        min_value, max_value, mean_value = get_min_max_mean(df_cpu, ' CPU', start_time, end_time)
        add_min_max_mean_columns(df_timestamp, 'CPU', i, min_value, max_value, mean_value)
        
        min_value, max_value, mean_value = get_min_max_mean(df_mem_cpu, ' USED', start_time, end_time)
        add_min_max_mean_columns(df_timestamp, 'MEM_CPU', i, min_value, max_value, mean_value)
                
    return df_timestamp
  
def sumarise_df_gpu(df_cpu, df_mem_cpu, df_gpu, df_mem_gpu, df_timestamp):
    df_cpu['TIMESTAMP'] = df_cpu['TIMESTAMP'].astype(int)
    df_mem_cpu['TIMESTAMP'] = df_mem_cpu['TIMESTAMP'].astype(int)
    df_gpu['TIMESTAMP'] = df_gpu['TIMESTAMP'].astype(int)
    df_mem_gpu['TIMESTAMP'] = df_mem_gpu['TIMESTAMP'].astype(int)
    df_timestamp['START_TIME'] = df_timestamp['START_TIME'].astype(str).str[:10].astype(int)
    df_timestamp['END_TIME'] = df_timestamp['END_TIME'].astype(str).str[:10].astype(int)

    for i, r in df_timestamp.iterrows():
        start_time = r['START_TIME']
        end_time = r['END_TIME']
        
        min_value, max_value, mean_value = get_min_max_mean(df_cpu, ' CPU', start_time, end_time)
        add_min_max_mean_columns(df_timestamp, 'CPU', i, min_value, max_value, mean_value)
        
        min_value, max_value, mean_value = get_min_max_mean(df_mem_cpu, ' USED', start_time, end_time)
        add_min_max_mean_columns(df_timestamp, 'MEM_CPU', i, min_value, max_value, mean_value)

        min_value, max_value, mean_value = get_min_max_mean(df_gpu, ' GPU', start_time, end_time)
        add_min_max_mean_columns(df_timestamp, 'GPU', i, min_value, max_value, mean_value)
        
        min_value, max_value, mean_value = get_min_max_mean(df_mem_gpu, ' USED', start_time, end_time)
        add_min_max_mean_columns(df_timestamp, 'MEM_GPU', i, min_value, max_value, mean_value)
        
    return df_timestamp


In [9]:
def get_subdirs(path):
    subdirs = []
    for subdir in os.listdir(path):
        if os.path.isdir(os.path.join(path, subdir)):
            subdirs.append(subdir)
    return subdirs


dir_names = get_subdirs(maindir)
for dir in dir_names:
    try:
        df_timestamp = pd.read_csv(f'{maindir}/{dir}/timestamp.csv')

        if execution == "CPU":
            df_cpu = pd.read_csv(f'{maindir}/{dir}/cpu.csv')
            df_mem_cpu = pd.read_csv(f'{maindir}/{dir}/mem.csv')
            df_timestamp = sumarise_df_cpu(df_cpu, df_mem_cpu, df_timestamp)

        if execution == "GPU":
            df_cpu = pd.read_csv(f'{maindir}/{dir}/cpu.csv')
            df_mem_cpu = pd.read_csv(f'{maindir}/{dir}/mem.csv')
            df_gpu = pd.read_csv(f'{maindir}/{dir}/gpu.csv')
            df_mem_gpu = pd.read_csv(f'{maindir}/{dir}/mem-gpu.csv')
            df_mem_gpu = df_mem_gpu[df_mem_gpu[' COMMAND'] == '  ./src/main']
            df_timestamp = sumarise_df_gpu(df_cpu, df_mem_cpu, df_gpu, df_mem_gpu, df_timestamp)

        df_timestamp.to_csv(f'{maindir}/{dir}/timestamp.csv', index=False)

    except Exception as e:
        print(f"Error on {dir}", e)
        pass

Error on experimento25 No columns to parse from file


In [10]:
def get_subdirs(path):
    subdirs = []
    for subdir in os.listdir(path):
        if os.path.isdir(os.path.join(path, subdir)):
            subdirs.append(subdir)
    return subdirs


df = pd.DataFrame()
dir_names = get_subdirs(f"{maindir}")
for dir in dir_names:
    try:
        df_timestamp = pd.read_csv(f'{maindir}/{dir}/timestamp.csv')
        df_timestamp.insert(0, 'directory', dir)
        df = pd.concat([df, df_timestamp], axis=0)
    except Exception as e:
        print(f"Error on {dir}", e)
        pass

df.sort_values(by=['directory'], inplace=True)
df.reset_index(drop=True, inplace=True)
df.to_csv(f'{maindir}/timestamp.csv', index=False)

Error on experimento25 No columns to parse from file
